In [36]:
import requests
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import datetime, timedelta
import time

api_key = 'Dmz0uotXzXaeWWQBx2nDWBmavU96iT8c'
base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'

end_date = datetime.now()
start_date = end_date - timedelta(days=365)
page = 0

# Define the search parameters
search_params = {
    'api-key': api_key,
    'q': 'IBM',
    'begin_date': start_date.strftime('%Y%m%d'),
    'end_date': end_date.strftime('%Y%m%d'),
    'page': page
}
sid = SentimentIntensityAnalyzer()


# Make the request to the NYTimes API
response = requests.get(base_url, params=search_params)
data = response.json()

# Extract relevant information from the response
articles = data['response']['docs']

# Initialize the sentiment analyzer

# Create a pandas dataframe to store the data
columns = ['headline', 'snippet', 'date', 'compound_sentiment']
df = pd.DataFrame(columns=columns)

# Iterate through the articles and calculate sentiment
headlines = []
snippets = []
dates = []
compound_sentiments = []

hits = data['response']['meta']['hits']

while page * 10 < (hits > 1000 and 1000 or hits):
    print(f"Processing page: {page}")
    for article in articles:
        headline = article['headline']['main']
        snippet = article['snippet']
        date = article['pub_date']
        
        # Calculate compound sentiment using VADER
        sentiment_scores = sid.polarity_scores(f"{headline}. {snippet}")
        compound_sentiment = sentiment_scores['compound']
        
        # # Append data to the dataframe
        # df = pd.concat((df, pd.DataFrame({
        #     'headline': headline,
        #     'snippet': snippet,
        #     'date': date,
        #     'compound_sentiment': compound_sentiment
        # })), ignore_index=True)
        headlines.append(headline)
        snippets.append(snippet)
        dates.append(date)
        compound_sentiments.append(compound_sentiment)
    
    page += 1
    search_params['page'] = page
    time.sleep(13)
    response = requests.get(base_url, params=search_params)
    data = response.json()
    articles = data['response']['docs']

Processing page: 0
Processing page: 1
Processing page: 2
Processing page: 3
Processing page: 4
Processing page: 5
Processing page: 6
Processing page: 7
Processing page: 8
Processing page: 9
Processing page: 10
Processing page: 11
Processing page: 12


In [37]:
df = pd.DataFrame({
    'headline': headlines,
    'snippet': snippets,
    'date': dates,
    'compound_sentiment': compound_sentiments
})

# Display the dataframe
df

,headline,snippet,date,compound_sentiment
0,Big Companies Find a Way to Identify A.I. Data...,Mainstream companies have concerns about the l...,2023-11-30T11:00:13+0000,0.5574
1,The Pension: That Rare Retirement Benefit Gets...,As the downsides of 401(k)-style plans become ...,2023-11-24T10:02:14+0000,0.8834
2,Advertisers Flee X as Outcry Over Musk’s Endor...,"Disney, Apple, Paramount and Lionsgate halted ...",2023-11-17T22:50:43+0000,-0.7351
3,X Races to Contain Damage After Elon Musk Endo...,"IBM, a major advertiser on X, has pulled its s...",2023-11-16T23:26:48+0000,-0.2023
4,Advertisers Push Back at Social Media Firms ov...,"Companies, celebrities and influencers have ta...",2023-11-17T13:06:41+0000,-0.7650
...,...,...,...,...
122,Tech Start-Ups Try to Sell a Cautious Pentagon...,"Shield AI, a tech start-up, already has a dron...",2023-11-06T10:00:49+0000,0.6428
123,He’s the Bad Boy of Chess. But Did He Cheat?,"An American teenager, Hans Niemann, defeated M...",2022-12-04T10:00:30+0000,-0.6369
124,Transcript: Ezra Klein Interviews Demis Hassabis,"The July 11, 2023, episode of “The Ezra Klein ...",2023-07-11T14:23:13+0000,0.0000
125,Transcript: Ezra Klein Interviews Kelsey Piper,"The Mar. 21, 2023, episode of “The Ezra Klein ...",2023-03-21T15:15:28+0000,0.0000
